This notebook runs a pipeline with the following steps:
* Create a test Visibility dataset.
   * No visibility sample noise. Just testing that things are working exactly as expected.
   * Using the GLEAM sky model and a common everybeam station beam model.
   * Add complex Gaussian noise corruptions to station Jones matrices.
   * Write to disk in MSv2 format.
* Read MSv2 data into Visibility dataset.$^{*}$
* Predict model visibilities (using GLEAM and everybeam).$^{*}$
* Do bandpass calibration.$^{*}$
* Check for convergence.$^{*}$

\* xarray dataset map_blocks() is used to distribute frequency sub-bands across dask tasks.

The pipeline is called once with a simple model, gain corruptions and a gain solver,
then again with a more complicated sky and beam model,
then again with gain and leakage corruptions and a polarised solver.

In [1]:
from time import perf_counter

from ska_sdp_instrumental_calibration.processing_tasks.lsm import Component
from ska_sdp_instrumental_calibration.workflow.pipelines import (
    bandpass_calibration,
)

<frozen abc>:106: FutureWarning: xarray subclass Visibility should explicitly define __slots__


In [2]:
gleamfile = "/data/EOS_1/mit183/gleamegc.dat"
eb_ms = "/data/EOS_1/mit183/SKA/SP-4626/OSKAR_MOCK.ms"
eb_coeffs = "/data/EOS_1/mit183/SKA/SP-4626/ska-sdp-func-everybeam/coeffs"

In [3]:
# Simple bandpass calibration of X and Y gains with a user-defined sky model
t0 = perf_counter()
bandpass_calibration.run(
    {
        "lsm": [
            Component(
                name="comp1", RAdeg=20.0, DEdeg=-30.0, flux=1.0, alpha=0.0
            ),
            Component(
                name="comp2", RAdeg=19.5, DEdeg=-29.9, flux=0.1, alpha=0.0
            ),
        ],
        "beam_type": "none",
        "ms_name": "demo.ms",
        "hdf5_name": "demo.hdf5",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2025-02-05 14:55:21,863 - workflow.pipeline_config - INFO - Simulating demo MSv2 input
2025-02-05 14:55:21,864 - workflow.pipeline_config - INFO - Predicting visibilities without a beam
2025-02-05 14:55:21,864 - workflow.pipeline_config - INFO - Generating a demo MSv2 Measurement Set demo.ms
2025-02-05 14:55:21,879 - workflow.utils - INFO - Using AA1-Low with 20 stations
2025-02-05 14:55:22,431 - processing_tasks.predict - INFO - No beam model used in predict
2025-02-05 14:55:22,584 - workflow.utils - INFO - Applying direction-independent gain corruptions
2025-02-05 14:55:23,049 - pipeline.bandpass_calibration - INFO - Starting pipeline with 16-channel chunks
2025-02-05 14:55:23,050 - pipeline.bandpass_calibration - INFO - No dask cluster supplied. Using LocalCluster
2025-02-05 14:55:24,560 - pipeline.bandpass_calibration - INFO - Setting calibration in 16-channel chunks
2025-02-05 14:55:24,561 - pipeline.bandpass_calibration - INFO - end_to_end_subbands = true
2025-02-05 14:55:24,793 

pipeline ran in 4.7 seconds


In [4]:
# Simple bandpass calibration of X and Y gains with GLEAM and EveryBeam
t0 = perf_counter()
bandpass_calibration.run(
    {
        # Required external data
        "gleamfile": gleamfile,
        "eb_ms": eb_ms,
        "eb_coeffs": eb_coeffs,
        # Other parameters
        "ms_name": "demo.ms",
        "hdf5_name": "demo.hdf5",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2025-02-05 14:55:26,611 - workflow.pipeline_config - INFO - Simulating demo MSv2 input
2025-02-05 14:55:26,612 - workflow.pipeline_config - INFO - Initialising the EveryBeam telescope model with /data/EOS_1/mit183/SKA/SP-4626/OSKAR_MOCK.ms
2025-02-05 14:55:26,612 - workflow.pipeline_config - INFO - Generating a demo MSv2 Measurement Set demo.ms
2025-02-05 14:55:26,613 - workflow.pipeline_config - INFO - Generating LSM for simulation with:
2025-02-05 14:55:26,614 - workflow.pipeline_config - INFO -  - Catalogue file: /data/EOS_1/mit183/gleamegc.dat
2025-02-05 14:55:26,614 - workflow.pipeline_config - INFO -  - Search radius: 5.0 deg
2025-02-05 14:55:26,615 - workflow.pipeline_config - INFO -  - Flux limit: 1 Jy
2025-02-05 14:55:27,196 - processing_tasks.lsm - INFO - extracted 37 GLEAM components
2025-02-05 14:55:27,197 - workflow.pipeline_config - INFO - LSM: found 37 components
2025-02-05 14:55:27,206 - workflow.utils - INFO - Using AA1-Low with 20 stations
2025-02-05 14:55:27,237 - pr

Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2025-02-05 14:55:30,589 - workflow.utils - INFO - Applying direction-independent gain corruptions
2025-02-05 14:55:31,225 - pipeline.bandpass_calibration - INFO - Starting pipeline with 16-channel chunks
2025-02-05 14:55:31,225 - pipeline.bandpass_calibration - INFO - No dask cluster supplied. Using LocalCluster
2025-02-05 14:55:32,002 - pipeline.bandpass_calibration - INFO - Generating LSM for predict with:
2025-02-05 14:55:32,004 - pipeline.bandpass_calibration - INFO -  - Catalogue file: /data/EOS_1/mit183/gleamegc.dat
2025-02-05 14:55:32,004 - pipeline.bandpass_calibration - INFO -  - Search radius: 5.0 deg
2025-02-05 14:55:32,005 - pipeline.bandpass_calibration - INFO -  - Flux limit: 1 Jy
2025-02-05 14:55:32,744 - processing_tasks.lsm - INFO - extracted 37 GLEAM components
2025-02-05 14:55:32,745 - pipeline.bandpass_calibration - INFO - LSM: found 37 components
2025-02-05 14:55:32,745 - pipeline.bandpass_calibration - INFO - Setting calibration in 16-channel chunks
2025-02-05 14:

Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2025-02-05 14:55:36,038 - pipeline.bandpass_calibration - INFO - Writing solutions to demo.hdf5
2025-02-05 14:55:36,048 - pipeline.bandpass_calibration - INFO - Checking results
2025-02-05 14:55:36,050 - pipeline.bandpass_calibration - INFO - Convergence checks passed


pipeline ran in 9.8 seconds


In [5]:
# Bandpass calibration with leakage
t0 = perf_counter()
bandpass_calibration.run(
    {
        # Required external data
        "gleamfile": gleamfile,
        "eb_ms": eb_ms,
        "eb_coeffs": eb_coeffs,
        # Other parameters
        "ms_name": "demo.ms",
        "hdf5_name": "demo.hdf5",
        "gains": True,
        "leakage": True,
        "solver": "normal_equations",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2025-02-05 14:55:36,452 - workflow.pipeline_config - INFO - Simulating demo MSv2 input
2025-02-05 14:55:36,453 - workflow.pipeline_config - INFO - Initialising the EveryBeam telescope model with /data/EOS_1/mit183/SKA/SP-4626/OSKAR_MOCK.ms
2025-02-05 14:55:36,454 - workflow.pipeline_config - INFO - Generating a demo MSv2 Measurement Set demo.ms
2025-02-05 14:55:36,455 - workflow.pipeline_config - INFO - Generating LSM for simulation with:
2025-02-05 14:55:36,455 - workflow.pipeline_config - INFO -  - Catalogue file: /data/EOS_1/mit183/gleamegc.dat
2025-02-05 14:55:36,456 - workflow.pipeline_config - INFO -  - Search radius: 5.0 deg
2025-02-05 14:55:36,456 - workflow.pipeline_config - INFO -  - Flux limit: 1 Jy
2025-02-05 14:55:37,039 - processing_tasks.lsm - INFO - extracted 37 GLEAM components
2025-02-05 14:55:37,040 - workflow.pipeline_config - INFO - LSM: found 37 components
2025-02-05 14:55:37,049 - workflow.utils - INFO - Using AA1-Low with 20 stations
2025-02-05 14:55:37,080 - pr

Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2025-02-05 14:55:40,415 - workflow.utils - INFO - Applying direction-independent gain corruptions
2025-02-05 14:55:40,417 - workflow.utils - INFO - Applying direction-independent leakage corruptions
2025-02-05 14:55:40,995 - pipeline.bandpass_calibration - INFO - Starting pipeline with 16-channel chunks
2025-02-05 14:55:40,996 - pipeline.bandpass_calibration - INFO - No dask cluster supplied. Using LocalCluster
2025-02-05 14:55:41,782 - pipeline.bandpass_calibration - INFO - Generating LSM for predict with:
2025-02-05 14:55:41,784 - pipeline.bandpass_calibration - INFO -  - Catalogue file: /data/EOS_1/mit183/gleamegc.dat
2025-02-05 14:55:41,785 - pipeline.bandpass_calibration - INFO -  - Search radius: 5.0 deg
2025-02-05 14:55:41,786 - pipeline.bandpass_calibration - INFO -  - Flux limit: 1 Jy
2025-02-05 14:55:42,422 - processing_tasks.lsm - INFO - extracted 37 GLEAM components
2025-02-05 14:55:42,423 - pipeline.bandpass_calibration - INFO - LSM: found 37 components
2025-02-05 14:55:42

Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2025-02-05 14:55:46,726 - pipeline.bandpass_calibration - INFO - Writing solutions to demo.hdf5
2025-02-05 14:55:46,739 - pipeline.bandpass_calibration - INFO - Checking results
2025-02-05 14:55:46,741 - pipeline.bandpass_calibration - INFO - Convergence checks passed


pipeline ran in 10.7 seconds
